## Get the github and import functions

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/Paperspace/DataAugmentationForObjectDetection

In [1]:
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
import os

In [2]:
!pwd
%cd DataAugmentationForObjectDetection

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch
/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/DataAugmentationForObjectDetection


In [3]:
from data_aug.data_aug import *
import cv2 
import pickle as pkl
import numpy as np 
import matplotlib.pyplot as plt

In [4]:
%cd ..

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch


## Generate yflip images

In [5]:
#need to have bbox in the right format
import xml.etree.ElementTree as ET
import os
def read_content(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    list_with_all_boxes = []
    list_with_all_labels = []
    for boxes in root.iter('object'):
        filename = root.find('filename').text
        label = boxes.find('name').text
        ymin, xmin, ymax, xmax = None, None, None, None
        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)
        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_labels.append(label)
        list_with_all_boxes.append(list_with_single_boxes)
    return list_with_all_labels,list_with_all_boxes

In [6]:
#the folder before aug is called batch2BfAug
img_bf_aug_test = []
from glob import glob
%cd /Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2BfAug/
for file_path in glob('/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2BfAug/*.xml'):
    single_image =[]
    image_filename = os.path.basename(file_path)
    #print(image_filename)
    labels,boxes = read_content(image_filename)
    single_image.append(image_filename)
    single_image.append(labels)
    single_image.append(boxes)
    img_bf_aug_test.append(single_image)
#print(gt_train_labels[1])
print(len(img_bf_aug_test))
%cd ..

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2BfAug
1168
/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4


In [7]:
cnt=0
for i in img_bf_aug_test:
    if i[1]==[]:
        cnt+=1
print('number of empty label images are',cnt)

number of empty label images are 0


In [8]:
#append name with ".jpg" and no_'.xml'& no_".jpg"
import imageio
for i in img_bf_aug_test:
    i.append(i[0].replace('.xml','.jpg'))
    i.append(i[0].replace('.xml',''))
for j in img_bf_aug_test[:2]:
    print(j)

['39.74551276--104.98828359999999-135.xml', ['lamp'], [[285, 175, 304, 425]], '39.74551276--104.98828359999999-135.jpg', '39.74551276--104.98828359999999-135']
['39.75050027--104.9852889-315.xml', ['lamp'], [[1, 66, 225, 487]], '39.75050027--104.9852889-315.jpg', '39.75050027--104.9852889-315']


In [9]:
img_af_aug=[]
%cd /Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2AfAug/
#cnt=0
for j in img_bf_aug_test:
    for file_path in glob('/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2AfAug/*.jpg'):
        single_img_af_aug =[]
        image_filename = os.path.basename(file_path)
        if j[3] == image_filename:
            #print(cnt,j[3],j[2])
            bboxes = np.array(j[2]).astype(np.float64)
            img = cv2.imread(image_filename)[:,:,::-1]
            hor_flip = RandomHorizontalFlip(1)  
            img, bboxes = hor_flip(img, bboxes)
            imageio.imwrite(j[4]+str('_yflip.jpg'), img)
            single_img_af_aug.append(j[4]+str('_yflip.jpg'))
            single_img_af_aug.append(j[1][::-1])
            single_img_af_aug.append(bboxes.tolist())
            img_af_aug.append(single_img_af_aug)
            #cnt+=1

/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2AfAug


In [ ]:
# img_af_aug=[]
# %cd /Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2AfAug/
# for file_path in glob('/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2AfAug/*.jpg'):
#     single_img_af_aug =[]
#     image_filename = os.path.basename(file_path)
#     for j in img_bf_aug_test:
#         if j[3] == image_filename:
#             print(j[3],j[2])
#             bboxes = np.array(j[2]).astype(np.float64)
#             img = cv2.imread(image_filename)[:,:,::-1]
#             hor_flip = RandomHorizontalFlip(1)  
#             img, bboxes = hor_flip(img, bboxes)
#             imageio.imwrite(j[4]+str('_yflip.jpg'), img)
#             single_img_af_aug.append(j[4]+str('_yflip.jpg'))
#             single_img_af_aug.append(j[1][::-1])
#             single_img_af_aug.append(bboxes.tolist())
#             img_af_aug.append(single_img_af_aug)
#img_af_aug

## Generate XML file from bbox

In [11]:
#change format to use below function
new_img_af_aug=[]
for img in img_af_aug:
    single_img=[]
    single_img.append(img[0])
    for i in range(len(img[1])):
        temp = [img[1][i], img[2][i]]
        single_img.append(temp)
    new_img_af_aug.append(single_img)
new_img_af_aug[:5]

[['39.74551276--104.98828359999999-135_yflip.jpg',
  ['lamp', [336.0, 175.0, 355.0, 425.0]]],
 ['39.75050027--104.9852889-315_yflip.jpg',
  ['lamp', [415.0, 66.0, 639.0, 487.0]]],
 ['39.74751024--104.9978648-315_yflip.jpg',
  ['fh', [161.0, 119.0, 279.0, 447.0]],
  ['lamp', [419.0, 436.0, 444.0, 476.0]]],
 ['39.74180225--104.9913497-135_yflip.jpg',
  ['lamp', [368.0, 137.0, 404.0, 448.0]]],
 ['39.74434244--104.98805420000001-135_yflip.jpg',
  ['lamp', [515.0, 318.0, 578.0, 512.0]],
  ['lamp', [182.0, 315.0, 231.0, 511.0]]]]

In [13]:
len(new_img_af_aug)

1168

In [15]:
#code is to create XML file for yflip images
def create_root(filename, width, height):
    root = ET.Element("annotations")
    ET.SubElement(root, "folder").text = "batch2AfAug"
    ET.SubElement(root, "filename").text = filename #"{}.jpg".format(file_prefix)
    ET.SubElement(root,"path").text = "/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2AfAug"
    source = ET.SubElement(root,"source")
    ET.SubElement(source,"database").text = "Unknown"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    ET.SubElement(root,"segmented").text = "0"
    return root


def create_object_annotation(root, voc_label):
    #for voc_label in voc_labels:
    obj = ET.SubElement(root, "object")
    ET.SubElement(obj, "name").text = voc_label[0]
    ET.SubElement(obj, "pose").text = "Unspecified"
    ET.SubElement(obj, "truncated").text = str(0)
    ET.SubElement(obj, "difficult").text = str(0)
    bbox = ET.SubElement(obj, "bndbox")
    ET.SubElement(bbox, "xmin").text = str(voc_label[1][0])
    ET.SubElement(bbox, "ymin").text = str(voc_label[1][1])
    ET.SubElement(bbox, "xmax").text = str(voc_label[1][2])
    ET.SubElement(bbox, "ymax").text = str(voc_label[1][3])
    return root

DESTINATION_DIR = "/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/batch2AfAug"
width=640
height=640
for image in new_img_af_aug:
    filename = image[0]
    root = create_root(filename, width, height)
    for j in range(1,len(image)):
        root = create_object_annotation(root, image[j])
    tree = ET.ElementTree(root)
    update_filename = filename.replace('.jpg','')
    tree.write("{}/{}.xml".format(DESTINATION_DIR, update_filename))


## Random split the filename to 70/30 two list

In [31]:
image_name=[]
for i in new_img_af_aug:
    image_name.append((i[0]))
len(image_name)

1168

In [32]:
# we should do a 70/30 split
1168*0.7

817.5999999999999

In [33]:
random.shuffle(image_name)

train_names = image_name[:818]
test_names = image_name[818:]
print(len(train_names))
print(len(test_names))


818
350


In [41]:
for i in test_names[:8]:
    print(i)

39.74468424--104.9910852-315_yflip.jpg
39.74163155--104.99896389999999-135_yflip.jpg
39.74030936--104.99152409999999-135_yflip.jpg
39.74315057--104.9896078-135_yflip.jpg
39.75197523--104.99033429999999-315_yflip.jpg
39.74618427--104.9891511-315_yflip.jpg
39.7492102--104.99742479999999-315_yflip.jpg
39.74934789--104.9885354-135_yflip.jpg


In [38]:
# each name has 4 files
# (1) .jpg
# (2) .xml
# (3) _yflip.jpg
# (4) _yflip.xml

full_test_name=[]
for test_name in test_names:
    full_test_name.append(test_name)
    full_test_name.append(test_name.replace('.jpg','.xml'))
    full_test_name.append(test_name.replace('_yflip.jpg','.jpg'))
    full_test_name.append(test_name.replace('_yflip.jpg','.xml'))
    
print('length of the files need to move to test folder are ', len(full_test_name))

length of the files need to move to test folder are  1400


In [40]:
full_test_name[:8]

['39.74468424--104.9910852-315_yflip.jpg',
 '39.74468424--104.9910852-315_yflip.xml',
 '39.74468424--104.9910852-315.jpg',
 '39.74468424--104.9910852-315.xml',
 '39.74163155--104.99896389999999-135_yflip.jpg',
 '39.74163155--104.99896389999999-135_yflip.xml',
 '39.74163155--104.99896389999999-135.jpg',
 '39.74163155--104.99896389999999-135.xml']

In [42]:
import shutil, os
for f in full_test_name:
    shutil.move(f, "/Users/KirbyMac/Desktop/BerkeleyClass/W210/POC1/FourBatch/batch4/test")